# set ENV

In [1]:
import numpy as np
import pandas as pd
import pickle

pd.options.display.max_rows = None
pd.options.display.max_columns = None

In [2]:
from sklearn import linear_model
import scipy.stats as stats

class LogisticRegression_with_p_values:

    
    def __init__(self, *args, **kwargs):
        self.model = linear_model.LogisticRegression(*args, **kwargs)
    def fit(self, X,y):
        self.model.fit(X,y)
        #calculate P-values
        denom = (2.0 * (1.0 + np.cosh(self.model.decision_function(X))))
        denom = np.tile(denom,(X.shape[1],1)).T
        F_ij = np.dot((X / denom).T,X)
        Cramer_Rao = np.linalg.inv(F_ij)
        sigma_estimates = np.sqrt(np.diagonal(Cramer_Rao))
        z_scores = self.model.coef_[0] / sigma_estimates
        p_values = [stats.norm.sf(abs(x)) * 2 for x in z_scores]
        #
        self.coef_ = self.model.coef_
        self.intercept_ = self.model.intercept_
        self.p_values = p_values
        self.feature_name = X.columns.values
        
    def summary(self):
        tb = pd.DataFrame(columns= ['feature'], data = self.feature_name)
        tb['coefficient'] = np.transpose(self.coef_)
        # revalue the index to make the df index starts from 1, like in R
        tb.index = tb.index + 1
        # add the Intercept in index 0
        tb.loc[0] = ['Intercept', self.intercept_[0]]
        #sort idx
        tb = tb.sort_index() 
        # p_values
        tb['p_value'] = np.append(np.nan, np.array(self.p_values))
        
        return tb

In [3]:
from sklearn import linear_model
from sklearn.metrics import mean_squared_error, r2_score
import scipy.stats as stats

class LinearRegression(linear_model.LinearRegression):
    
    def __init__(self, fit_intercept=True, normalize=False, copy_X=True,
                 n_jobs=1):
        self.fit_intercept = fit_intercept
        self.normalize = normalize
        self.copy_X = copy_X
        self.n_jobs = n_jobs
    
    def fit(self, X, y, n_jobs=1):
        self = super(LinearRegression, self).fit(X, y, n_jobs)
        sse = np.sum((self.predict(X) - y) ** 2, axis=0) / float(X.shape[0] - X.shape[1])
        se = np.array([np.sqrt(np.diagonal(sse * np.linalg.inv(np.dot(X.T, X))))])
        self.t = self.coef_ / se # t-score
        self.p = np.squeeze(2 * (1 - stats.t.cdf(np.abs(self.t), y.shape[0] - X.shape[1]))) #p-value
        
        #summary
        tb = pd.DataFrame(columns= ['feature'], data = X.columns.values)
        tb['coefficient'] = np.transpose(self.coef_)
        ##reindex, add intercept
        tb.index = tb.index + 1
        tb.loc[0] = ['intercept', self.intercept_]
        tb = tb.sort_index()
        ## p-values
        p_value = self.p # p for each coefficient
        p_value = np.append(np.nan, np.array(p_value)) # p for intercept, a nan value
        tb['p_value'] = p_value.round(4)
        
        self.summary = tb
    
    # predict and result
    def predict_n_result(self, X_test, Y_test):
        pred = self.predict(X_test)
        actual = np.array(Y_test)
        residual = actual - pred
        result = pd.DataFrame(columns= ['actual', 'pred', 'residual'],
                              data = np.transpose([actual, pred, residual]))
        return result

In [4]:
# read files
data = pd.read_csv('processed_data/data_processed.csv', index_col= 0)
pd_feature = pd.read_csv('model_result/pd_feature_after_selected.csv', index_col= 0)['feature'].to_list()
lgd_ead_feature = pd.read_csv('processed_data/lgd_ead_feature.csv', index_col= 0)['feature'].to_list()
pd_reference = pd.read_csv('model_result/pd_reference_after_selected.csv', index_col= 0)['reference'].to_list()
reference = pd.read_csv('processed_data/reference_categories.csv', index_col= 0)
dependent = pd.read_csv('processed_data/dependent_name.csv', index_col= 0)
score_card = pd.read_csv('model_result/score_card.csv', index_col= 0)

# models
pd_model = pickle.load(open('model_result/pd_model.sav', 'rb'))
lgd_s1_model = pickle.load(open('model_result/lgd_model_stage_1.sav', 'rb'))
lgd_s2_model = pickle.load(open('model_result/lgd_model_stage_2.sav', 'rb'))
ead_model = pickle.load(open('model_result/ead_model.sav', 'rb'))


/Users/warren/opt/anaconda3/envs/credit_risk/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3145: DtypeWarning: Columns (20,48) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [5]:
# prepare data and list
## reference list
lgd_ead_reference = reference.loc[reference['reference'].isin(lgd_ead_feature),'reference'].to_list()
dependent_dict = {i:j for i,j in zip(dependent['model'], dependent['dependent'])}


##slice data
data_PD = data.copy()[pd_feature].drop(pd_reference, axis = 1)
data_lgd_ead = data.copy()[lgd_ead_feature].drop(lgd_ead_reference, axis = 1)

#fill NA
data_lgd_ead.mths_since_last_delinq_numeric.fillna(0, inplace = True)
data_lgd_ead.mths_since_last_record_numeric.fillna(0, inplace = True)

# Predict

## PD, LGD, EAD

In [6]:
# PD
PD = pd_model.model.predict_proba(data_PD)[:,0]
#LGD
pred_recoverable = lgd_s1_model.model.predict(data_lgd_ead)
pred_recovery_rate = lgd_s2_model.predict(data_lgd_ead)
LGD = 1 - ( pred_recoverable * pred_recovery_rate )
LGD= np.where(LGD>1, 1, LGD)
LGD= np.where(LGD<0, 0, LGD)
#EAD
pred_CCF = ead_model.predict(data_lgd_ead)
pred_CCF= np.where(pred_CCF>1, 1, pred_CCF)
pred_CCF= np.where(pred_CCF<0, 0, pred_CCF)
EAD = data['funded_amnt_numeric'] * pred_CCF


## EL

In [7]:
EL = PD*LGD*EAD

EL.head(20)

0      420.028003
1      472.158641
2      449.786433
3     1267.986255
4      321.106228
5      345.199858
6     1627.690654
7      516.981283
8     1836.218362
9      779.262945
10     754.643189
11     907.510845
12    1150.866939
13     208.778488
14     578.649291
15     197.381659
16    1176.460960
17      60.203133
18     312.305413
19     292.424985
Name: funded_amnt_numeric, dtype: float64

## Credit score

In [8]:
#prepare
score_card.reset_index(drop = True)
# feature index
score_feature = score_card['feature'].to_list()

#score card
scores = score_card['credit_score'].values
scores = scores.reshape((scores.shape[0],1)) # for dot use

#data for calculate score
score_data = data.copy()
score_data.insert(0,'Intercept', 1)
score_data = score_data[score_feature]
score_data.head()

,Intercept,addr_state:AR_MI_PA_OH_MN,addr_state:CA,addr_state:GA_WA_OR,addr_state:IL_CT,addr_state:KS_SC_CO_VT_AK_MS,addr_state:ND_NE_IA_NV_FL_HI_AL,addr_state:NM_VA,addr_state:NY,addr_state:OK_TN_MO_LA_MD_NC,addr_state:RI_MA_DE_SD_IN,addr_state:TX,addr_state:UT_KY_AZ_NJ,addr_state:WI_MT,addr_state:WV_NH_WY_DC_ME_ID,annual_inc:100.0K-120K,annual_inc:120.0K-140K,annual_inc:20.0K-50K,annual_inc:50.0K-60K,annual_inc:60.0K-70K,annual_inc:70.0K-80K,annual_inc:80.0K-90K,annual_inc:90.0K-100K,annual_inc:<=20K,annual_inc:>=140K,dti:1.4-3.5,dti:10.5-16.1,dti:16.1-20.3,dti:20.3-21.7,dti:21.7-22.4,dti:22.4-35,dti:3.5-7.7,dti:7.7-10.5,dti:<=1.4,dti:>=35.0,emp_length_int:2-5,emp_length_int:6-7,emp_length_int:8-10,emp_length_int:<=0,emp_length_int:=1,emp_length_int:>=10,home_ownership:MORTGAGE,home_ownership:OWN,home_ownership:RENT_OTHER_NONE_ANY,initial_list_status:f,initial_list_status:w,inq_last_6mths:1-2,inq_last_6mths:3-6,inq_last_6mths:<=0,inq_last_6mths:>=7,int_rate:12.026-15.74,int_rate:15.741-20.281,int_rate:9.548-12.025,int_rate:<9.548,int_rate:>=20.282,mths_since_earliest_cr_line:140.0-165,mths_since_earliest_cr_line:165.0-248,mths_since_earliest_cr_line:248.0-271,mths_since_earliest_cr_line:<140.0,mths_since_earliest_cr_line:>353,mths_since_issue_d:38.0-40,mths_since_issue_d:40.0-42,mths_since_issue_d:42.0-49,mths_since_issue_d:49.0-53,mths_since_issue_d:53.0-65,mths_since_issue_d:65.0-85,mths_since_issue_d:<=38,mths_since_issue_d:>=85.0,mths_since_last_delinq:0-3,mths_since_last_delinq:31-56,mths_since_last_delinq:4-30,mths_since_last_delinq:>=57,mths_since_last_delinq:Missing,mths_since_last_record:0-2,mths_since_last_record:21-31,mths_since_last_record:3-20,mths_since_last_record:32-80,mths_since_last_record:81-86,mths_since_last_record:>=86.0,mths_since_last_record:Missing,purpose:credit_card,purpose:debt_consolidation,purpose:educ__sm_b__ren_en__mov,purpose:major_purch__car__home_impr,purpose:oth__med__vacation__house__wedd,term_int:36,term_int:60,total_acc:28-51,total_acc:<=27,total_acc:>=52,total_rev_hi_lim:10.0K-20K,total_rev_hi_lim:20.0K-30K,total_rev_hi_lim:30.0K-40K,total_rev_hi_lim:40.0K-55K,total_rev_hi_lim:5.0K-10K,total_rev_hi_lim:55.0K-95K,total_rev_hi_lim:<=5K,total_rev_hi_lim:>=95K,verification_status:Not Verified,verification_status:Source Verified,verification_status:Verified
0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,1,1,0,1,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,1,1,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,1,0,0,0,1
1,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,1,0,0,1,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,1,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0,1,0
2,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,1,1,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,1,0,0,1,0,0,1,0,0,0,0,0,0,0,1,0,1,0,0
3,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,1,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,1,0,1,0,0,0,0,0,0,1,0,0,0,0,1,0
4,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,1,0,0,0,1,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,1,1,0,0,0,0,0,0,0,0,1,0,0,1,0


In [9]:
pred_credit_score = score_data.dot(scores)
pred_credit_score.head()

,0
0,592.0
1,546.0
2,526.0
3,536.0
4,565.0


# combine ALL

In [10]:
data_output = data[['member_id', 'funded_amnt_numeric','loan_status','recovery_rate','CCF']]
data_output.columns = ['member_id', 'funded_amnt','loan_status','recovery_rate','CCF']
data_output['Credit_Score'] = pred_credit_score
data_output['PD'] = PD
data_output['LGD'] = LGD
data_output['EAD'] = EAD
data_output['Expected_Loss'] = EL

<ipython-input-10-b89dbd06c993>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_output['Credit_Score'] = pred_credit_score
<ipython-input-10-b89dbd06c993>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_output['PD'] = PD
<ipython-input-10-b89dbd06c993>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/i

In [11]:
data_output.reset_index(drop = True)
data_output.head(30)

# antything before Credit_Score are "actual"

,member_id,funded_amnt,loan_status,recovery_rate,CCF,Credit_Score,PD,LGD,EAD,Expected_Loss
0,1296599,5000,Fully Paid,0.000000,0.000000,592.0,0.155799,0.919856,2930.843008,420.028003
1,1314167,2500,Charged Off,0.046832,0.817416,546.0,0.249916,0.907004,2082.980119,472.158641
2,1313524,2400,Fully Paid,0.000000,0.000000,526.0,0.305359,0.908850,1620.703294,449.786433
3,1277178,10000,Fully Paid,0.000000,0.000000,536.0,0.214548,0.909123,6500.809012,1267.986255
4,1311748,3000,Current,0.000000,0.255633,565.0,0.156001,0.900400,2286.053036,321.106228
5,1311441,5000,Fully Paid,0.000000,0.000000,620.0,0.112794,0.929797,3291.526915,345.199858
6,1304742,7000,Current,0.000000,0.269879,514.0,0.342117,0.901227,5279.143357,1627.690654
7,1288686,3000,Fully Paid,0.000000,0.000000,536.0,0.277329,0.920246,2025.699438,516.981283
8,1306957,5600,Charged Off,0.033761,0.971068,485.0,0.432271,0.903089,4703.682146,1836.218362
9,1306721,5375,Charged Off,0.050100,0.874701,563.0,0.209053,0.913042,4082.590572,779.262945


# save output

In [12]:
data_output.to_csv('final_report.csv')